In [29]:
import json
import os
import numpy as np
import math

In [148]:
path_train = 'IT_train.json'
path_test = 'IT_test.json'

with open(path_train, "r") as f:
    data_train = json.load(f)
    f.close()
    
with open(path_test, "r") as f:
    data_test = json.load(f)
    f.close()

In [133]:
# path_train = 'CL_train.json'
# path_test = 'CL_test.json'

# with open(path_train, "r") as f:
#     data_train = json.load(f)
#     f.close()
    
# with open(path_test, "r") as f:
#     data_test = json.load(f)
#     f.close()

In [68]:
def isNaN(string):
    return string != string

In [73]:
for dictionary in [data_train, data_test]:
    for case in dictionary.keys():
        for p in dictionary[case]['user_3_overall']:
            if(isNaN(p)):
                print(1)

In [149]:
total_sentences = 0
labels = []
for dic in [data_train, data_test]:
    for case in dic.keys():
        total_sentences += len(dic[case]['sentences'])
        for key in ['user_1_overall', 'user_2_overall', 'user_3_overall']:
            for label in dic[case][key]:
                labels.append(str(label))
labels = np.unique(np.array(labels))

In [151]:
labelstoidx = {}
count = 0
for label in labels:
    labelstoidx[label] = count
    count += 1

In [153]:
n = np.zeros((total_sentences, len(labels)))

In [154]:
start = 0
for dic in [data_train, data_test]:
    for case in dic.keys():
        for u1, u2, u3 in zip(dic[case]['user_1_overall'], dic[case]['user_2_overall'], dic[case]['user_3_overall']):
            n[start][labelstoidx[str(u1)]] += 1
            n[start][labelstoidx[str(u2)]] += 1
            n[start][labelstoidx[str(u3)]] += 1
            start += 1

In [156]:
## Calculate pj's

pj = np.zeros(n.shape[1])
n_raters = 3

for j in range(n.shape[1]):
    total = 0
    for i in range(n.shape[0]):
        total += n[i][j]
    pj[j] = total/(n_raters*n.shape[0])

In [157]:
pj

array([0.08609131, 0.05265614, 0.00033944, 0.18788187, 0.01366259,
       0.09602003, 0.03818737, 0.00067889, 0.09729294, 0.21741344,
       0.06008147, 0.03127122, 0.11842329])

In [158]:
## Calculate Pi's

Pi = np.zeros(n.shape[0])
n_raters = 3
for i in range(n.shape[0]):
    total = 0
    for j in range(n.shape[1]):
        total += n[i][j]*(n[i][j]-1)
    Pi[i] = total/((n_raters-1)*n_raters)

In [159]:
## Calculate P_dash

P_dash = 0
for i in range(n.shape[0]):
    P_dash += Pi[i]
P_dash /= n.shape[0]

In [160]:
## Calculate P_e

P_e = 0

for j in range(n.shape[1]):
    P_e += pj[j]*pj[j]

In [161]:
(P_dash-P_e)/(1-P_e)

0.5923612507457806

In [100]:
## ALL labels IT : 0.5923612507457806
## ALL labels CL : 0.8640880615553184

## For Compressed Labels

In [134]:
mapping = {}
mapping['ArgumentPetitioner'] = 'Argument'
mapping['ArgumentRespondent'] = 'Argument'
mapping['Fact'] = 'Fact'
mapping['Issue'] = 'Fact'
mapping['PrecedentNotReliedUpon'] = 'Precedent'
mapping['PrecedentReliedUpon'] = 'Precedent'
mapping['PrecedentOverruled'] = 'Precedent'
mapping['RatioOfTheDecision'] = 'RatioOfTheDecision'
mapping['Statute'] = 'Statute'
mapping['RulingByPresentCourt'] = 'RulingByPresentCourt'
mapping['RulingByLowerCourt'] = 'RulingByLowerCourt'

In [135]:
total_sentences = 0
labels = []
for dic in [data_train, data_test]:
    for case in dic.keys():
        for idx in range(len(dic[case]['sentences'])):
            poss = True
            for key in ['user_1_overall', 'user_2_overall', 'user_3_overall']:
                curr = str(dic[case][key][idx])
                if(curr == 'None' or curr == 'Dissent'):
                    poss = False
            if(poss):
                labels.append(mapping[str(dic[case]['user_1_overall'][idx])])
                labels.append(mapping[str(dic[case]['user_2_overall'][idx])])
                labels.append(mapping[str(dic[case]['user_3_overall'][idx])])
                total_sentences += 1
labels = np.unique(np.array(labels))

In [137]:
labelstoidx = {}
count = 0
for label in labels:
    labelstoidx[label] = count
    count += 1

In [139]:
n = np.zeros((total_sentences, len(labels)))

In [140]:
start = 0
for dic in [data_train, data_test]:
    for case in dic.keys():
        for idx in range(len(dic[case]['sentences'])):
            poss = True
            for key in ['user_1_overall', 'user_2_overall', 'user_3_overall']:
                curr = str(dic[case][key][idx])
                if(curr == 'None' or curr == 'Dissent'):
                    poss = False
            if(poss):
                n[start][labelstoidx[mapping[str(dic[case]['user_1_overall'][idx])]]] += 1
                n[start][labelstoidx[mapping[str(dic[case]['user_2_overall'][idx])]]] += 1
                n[start][labelstoidx[mapping[str(dic[case]['user_3_overall'][idx])]]] += 1
                start += 1

In [142]:
## Calculate pj's

pj = np.zeros(n.shape[1])
n_raters = 3

for j in range(n.shape[1]):
    total = 0
    for i in range(n.shape[0]):
        total += n[i][j]
    pj[j] = total/(n_raters*n.shape[0])

In [143]:
## Calculate Pi's

Pi = np.zeros(n.shape[0])
n_raters = 3
for i in range(n.shape[0]):
    total = 0
    for j in range(n.shape[1]):
        total += n[i][j]*(n[i][j]-1)
    Pi[i] = total/((n_raters-1)*n_raters)

In [144]:
## Calculate P_dash

P_dash = 0
for i in range(n.shape[0]):
    P_dash += Pi[i]
P_dash /= n.shape[0]

In [145]:
## Calculate P_e

P_e = 0

for j in range(n.shape[1]):
    P_e += pj[j]*pj[j]

In [146]:
(P_dash-P_e)/(1-P_e)

0.8690448461407186

In [147]:
## Compressed Labels IT : 0.6548985720011437
## Compressed Labels CL: 0.8690448461407186

## ALL labels IT : 0.5923612507457806
## ALL labels CL : 0.8640880615553184